<a href="https://colab.research.google.com/github/vladimirtuzovskiy/pythin-ITMO/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22PythonUrban_final_task_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Итоговый проект. Улицы. Объекты культурного наследия.

In [1]:
# TODO собрать установку всех необходимых модулей в одном месте
!pip install geopandas
!pip install osmnx
!pip install keplergl

In [2]:
# TODO собрать импорты всех модулей в одном месте
import geopandas as gpd
import osmnx as ox
import pandas as pd
from keplergl import KeplerGl

In [3]:
TILES = "CartoDB positron"  # Название подложки для карт

# TODO указать любой район Санкт-Петербург из OSM https://wiki.openstreetmap.org/wiki/RU:%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B
TERRITORY_NAME = 'Петроградский район, Санкт-Петербург'  # название территории для которой будут строиться слои

# TODO указать ссылку на файл из вашего github репозитория, которая начинается с https://raw.githubusercontent.com/
KGIOP_FILE_URL = "https://raw.githubusercontent.com/aeksei/PythonUrbanITMO2023/main/geojson_layers/kgiop_objects.geojson" # ссылка на слой с объектами культурного наследия
STREETS_FILE_URL = "https://raw.githubusercontent.com/aeksei/PythonUrbanITMO2023/main/geojson_layers/streets.geojson"  # ссылка на слой с улицами

## Территория

### Загрузка территории из OSM (Extract)

In [4]:
# TODO загрузить geodataframe с геометрией для территории TERRITORY_NAME
district = ox.geocode_to_gdf(TERRITORY_NAME)
district

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name
0,"POLYGON ((30.21013 59.97234, 30.21024 59.97166...",59.982776,59.945656,30.345415,30.210134,182050703,relation,1114905,59.964234,30.285801,boundary,administrative,10,0.378016,administrative,Petrogradsky District,"Petrogradsky District, округ Чкаловское, Saint..."


## Улицы

### Загрузка файла с улицами (Extract)

In [5]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
streets = gpd.read_file(STREETS_FILE_URL, mask=district)

streets

,name,geometry
0,Зоологический переулок,"LINESTRING (3373462.006 8388151.101, 3373511.9..."
1,Петропавловская улица,"LINESTRING (3374588.102 8392070.811, 3374590.5..."
2,Вяземский переулок,"LINESTRING (3372868.695 8394317.579, 3372874.7..."
3,Большая Зеленина улица,"LINESTRING (3372527.913 8390692.726, 3372517.0..."
4,Прожекторная улица,"LINESTRING (3369192.747 8393842.365, 3369185.4..."
...,...,...
1115,Введенская улица,"LINESTRING (3373088.896 8390163.981, 3373069.4..."
1116,1-я Берёзовая аллея,"LINESTRING (3372932.203 8395106.340, 3372911.4..."
1117,Каменноостровский проспект,"LINESTRING (3374130.535 8392406.031, 3374148.9..."
1118,улица Красного Курсанта,"LINESTRING (3370103.497 8391454.546, 3370065.2..."


### Обработка данных с улицами (Transform)

In [6]:
# TODO сгруппировать и объединить геометрии с одинаковыми названиями
streets = streets.dissolve(by="name")
streets

,geometry
name,
1-й Елагин мост,"LINESTRING (3369923.760 8394685.829, 3369717.1..."
1-я Берёзовая аллея,"MULTILINESTRING ((3372005.435 8394804.438, 337..."
11-й Каменноостровский мост,"LINESTRING (3371132.111 8394928.299, 3371110.3..."
14-й Каменноостровский мост,"LINESTRING (3370500.317 8394416.937, 3370526.5..."
19-й Каменноостровский мост,"LINESTRING (3372279.425 8395681.946, 3372287.3..."
...,...
улица Савиной,"MULTILINESTRING ((3367867.656 8391660.748, 336..."
улица Сперанского,"MULTILINESTRING ((3372509.066 8388716.582, 337..."
улица Чапаева,"MULTILINESTRING ((3376459.929 8390890.301, 337..."


### Сохранение слоя с улицами (Load)

In [7]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
rename_columns = {
    "name": "Название"
}
streets.rename(columns=rename_columns, inplace=True)
# TODO для того чтобы переименовать индекс, нужно обратиться и нему и вызвать от него метод rename (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.rename.html)
streets.head()

,geometry
name,
1-й Елагин мост,"LINESTRING (3369923.760 8394685.829, 3369717.1..."
1-я Берёзовая аллея,"MULTILINESTRING ((3372005.435 8394804.438, 337..."
11-й Каменноостровский мост,"LINESTRING (3371132.111 8394928.299, 3371110.3..."
14-й Каменноостровский мост,"LINESTRING (3370500.317 8394416.937, 3370526.5..."
19-й Каменноостровский мост,"LINESTRING (3372279.425 8395681.946, 3372287.3..."


In [8]:
# TODO сохранить слой в географической проекции в формате GeoJSON
streets.to_crs(4326).to_file('streets.geojson', driver='GeoJSON')

## Объекты культурного наследия

### Загрузка объектов культурного наследия (Extract)

In [9]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
kgiop_objects = gpd.read_file(KGIOP_FILE_URL, mask=district)

kgiop_objects

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry
0,4380,—,Дом Н.А. Бородулина,1901-1902,"Большой пр. П.С., 13; Пионерская ул., 4",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3372466.498 8389787.753)
1,4669,—,Дом А.Л. Лишневского,1912-1913,"Ленина ул., 41; Полозова ул., 28",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3372869.697 8391922.956)
2,4670,—,Здание Петроградской АТС,1920-е,"Ленина ул., 5",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3374015.286 8390951.787)
3,2360,Дом Каменноостровского товарищества для устрой...,Служебный корпус Литера Б с двумя соединительн...,1911-1912; 1912,"Санкт-Петербург , ул. Профессора Попова, д. 4,...",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-496 от 18.08.2014,Памятник,POINT (3374952.373 8393202.996)
4,4758,Особняк Ф.И. Андреевского,Ограда,около 1910,"Ольгина ул., 2",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3370369.127 8393973.351)
...,...,...,...,...,...,...,...,...,...
715,9553,—,Улица Красного Курсанта (большая Спасская). Уч...,—,"Красного Курсанта ул., 10",выявленный объект культурного наследия,Распоряжение КГИОП от 02.11.2018 № 447-р,Памятник,POINT (3371983.928 8389824.216)
716,9554,—,Мичуринская улица 1. Участок культурного слоя ...,—,"Мичуринская ул., 14",выявленный объект культурного наследия,Распоряжение КГИОП от 02.11.2018 № 448-р,Памятник,POINT (3375771.462 8389996.976)
717,9555,—,Провиантская улица (Мытный переулок) 1. Участо...,—,"Провиантская ул., 8",выявленный объект культурного наследия,Распоряжение КГИОП от 02.11.2018 № 449-р,Памятник,POINT (3373253.860 8388771.069)
718,9560,—,Колтовская слобода 2. Участок культурного слоя...,—,"Адмирала Лазарева наб., 20",выявленный объект культурного наследия,Распоряжение КГИОП от 21.12.2018 № 546-р,Памятник,POINT (3370663.901 8391938.524)


### Обработка объектов культурного наследия (Transform)

In [10]:
# TODO добавить два столбца lon и lat, в которых будут долгота и широта
kgiop_objects["lon"] = kgiop_objects.to_crs(3857).geometry.centroid.to_crs(4326).x
kgiop_objects["lat"] = kgiop_objects.to_crs(3857).geometry.centroid.to_crs(4326).y
drop_columns = ["id",
                "ensemble_name",
]
kgiop_objects.drop(columns=drop_columns, inplace=True)
kgiop_objects

,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry,lon,lat
0,Дом Н.А. Бородулина,1901-1902,"Большой пр. П.С., 13; Пионерская ул., 4",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3372466.498 8389787.753),30.295382,59.955278
1,Дом А.Л. Лишневского,1912-1913,"Ленина ул., 41; Полозова ул., 28",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3372869.697 8391922.956),30.299004,59.964880
2,Здание Петроградской АТС,1920-е,"Ленина ул., 5",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3374015.286 8390951.787),30.309295,59.960513
3,Служебный корпус Литера Б с двумя соединительн...,1911-1912; 1912,"Санкт-Петербург , ул. Профессора Попова, д. 4,...",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-496 от 18.08.2014,Памятник,POINT (3374952.373 8393202.996),30.317713,59.970635
4,Ограда,около 1910,"Ольгина ул., 2",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3370369.127 8393973.351),30.276541,59.974098
...,...,...,...,...,...,...,...,...,...
715,Улица Красного Курсанта (большая Спасская). Уч...,—,"Красного Курсанта ул., 10",выявленный объект культурного наследия,Распоряжение КГИОП от 02.11.2018 № 447-р,Памятник,POINT (3371983.928 8389824.216),30.291047,59.955442
716,Мичуринская улица 1. Участок культурного слоя ...,—,"Мичуринская ул., 14",выявленный объект культурного наследия,Распоряжение КГИОП от 02.11.2018 № 448-р,Памятник,POINT (3375771.462 8389996.976),30.325071,59.956219
717,Провиантская улица (Мытный переулок) 1. Участо...,—,"Провиантская ул., 8",выявленный объект культурного наследия,Распоряжение КГИОП от 02.11.2018 № 449-р,Памятник,POINT (3373253.860 8388771.069),30.302455,59.950705
718,Колтовская слобода 2. Участок культурного слоя...,—,"Адмирала Лазарева наб., 20",выявленный объект культурного наследия,Распоряжение КГИОП от 21.12.2018 № 546-р,Памятник,POINT (3370663.901 8391938.524),30.279189,59.964950


### Сохранение слоя с объектами культурного наследия (Load)

In [11]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
rename_columns = {
    "object_name": "Название",
    "occurrence_time": "Период создания",
    "object_location": "Адрес расположения",
    "historical_category": "Категория",
    "normative_act": "Нормативный акт",
    "object_type": "Тип",
}
kgiop_objects.rename(columns=rename_columns, inplace=True)
kgiop_objects.head()

,Название,Период создания,Адрес расположения,Категория,Нормативный акт,Тип,geometry,lon,lat
0,Дом Н.А. Бородулина,1901-1902,"Большой пр. П.С., 13; Пионерская ул., 4",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3372466.498 8389787.753),30.295382,59.955278
1,Дом А.Л. Лишневского,1912-1913,"Ленина ул., 41; Полозова ул., 28",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3372869.697 8391922.956),30.299004,59.964880
2,Здание Петроградской АТС,1920-е,"Ленина ул., 5",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3374015.286 8390951.787),30.309295,59.960513
3,Служебный корпус Литера Б с двумя соединительн...,1911-1912; 1912,"Санкт-Петербург , ул. Профессора Попова, д. 4,...",объект культурного наследия регионального знач...,Распоряжение КГИОП № 10-496 от 18.08.2014,Памятник,POINT (3374952.373 8393202.996),30.317713,59.970635
4,Ограда,около 1910,"Ольгина ул., 2",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3370369.127 8393973.351),30.276541,59.974098


In [12]:
# TODO сохранить слой в географической проекции в формате GeoJSON
kgiop_objects.to_crs(4326).to_file('kgiop_objects.geojson', driver='GeoJSON')

In [14]:
from google.colab import output
output.enable_custom_widget_manager()

In [15]:
map = KeplerGl(height=500)
map.add_data(data=kgiop_objects.copy(), name="ОКН")  # почему-то метод add_data изменяет геодатафрейм :( будем делать копию

map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'ОКН': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,…